### import

In [1]:
import cv2
#from sklearn.preprocessing import MinMaxScaler
import numpy as np
#import joblib
import tensorflow as tf
import scipy.linalg as la
import random
#from google.colab.patches import cv2_imshow
import copy
import time
from rdkit.Chem import AllChem, Draw
from rdkit import Chem
from PIL import Image
import math
# from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions
import py3Dmol
# from IPython.display import display
import matplotlib.pyplot as plt  # plt 用於顯示圖片
import matplotlib.image as mpimg  # mpimg 用於讀取圖片
import os
#import pathlib
#import gradio as gr
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
import filetype
from IPython.display import display, clear_output
from ipywidgets import IntSlider, Output

buttom所需

In [2]:
out = Output()

## 輸入檔案(步驟1)

輸入*檔案位置*與*數字*(使用一次輸入一張圖片或影片)

    (刪除:(存入檔案名稱進*inputfile*(例如:'image'))，因一張圖片使用一次函數)
    
    若是圖案就輸入檔案位置呈現圖案，之後resize成300X300像素，3維變4維是dst(刪除:(存入*dim4resizeimg*))
    
    若是其他的(也就是影片)就使用cutvideotoimage這個函數(該函數需要*數字*以儲存很多圖片檔案)，然後是dst(刪除:(存入*dim4resizeimg*))
    
輸出*str(kind.mime).split('/')[0]* 與 *dst*，分別是一個圖案的*檔案種類*與*檔案像素*。

ps裡面使用到了cutvideotoimage自定義函數

In [3]:

def input_file(file_path, input_number):
    #print(inputfile)
    fileURL = (file_path)
    kind = filetype.guess(fileURL)                                                    #抽出File類型
    #inputfile.append(str(kind.mime).split('/')[0])
    if str(kind.mime).split('/')[0]=='image':                                         #若輸入檔案是圖片
         # 顯示圖片
        img = mpimg.imread(file_path)                                                 # 讀取和程式碼處於同一目錄下的 i
        # 此時 img 是一個 np.array 了，可以對它進行任意處理
        plt.imshow(img)                                                               # 顯示圖片
        plt.axis('off')                                                               # 不顯示座標軸
        plt.show()
        # cv2_imshow(im)
        # 圖片壓縮成像素300*300
        im = copy.deepcopy(cv2.imread(file_path))                                      # 讀取和程式碼處於同一目錄下的 i
        # 此時 im 是一個 np.array 了，可以對它進行任意處理
        dst = copy.deepcopy(cv2.resize(im, (300, 300), interpolation=cv2.INTER_AREA))  # resize
        dst = copy.deepcopy(dst[np.newaxis, :])                                        # 3維變4維
        #dim4resizeimg.append(dst)
    else:                                                                              # 若輸入不是圖片(而是影片)
        #dim4resizeimg.append(cutvideotoimage(file_path, str(input_number)+'image', str(input_number)+'data'))
        dst=cutvideotoimage(file_path, str(input_number)+'image', str(input_number)+'data')
    return str(kind.mime).split('/')[0],dst

沒輸入

    從外部檔案輸入圖片位置
    
    使用到 input_file 函數輸入*檔案位置*與*數字*得到一張張*圖片種類*與*像素*與呈現
    
    每張存入*dim4resizeimg*與*inputfile*
    
輸出*inputfile*與*dim4resizeimg*，分別是*檔案種類*與*檔案像素*。dim4resizeimg=[檔案1四維, 檔案2四維]  影片與圖片的數量不同

ps裡面使用到了 input_file自定義函數

In [4]:
def frf(ppo):
    dim4resizeimg=[]
    inputfile = []
    root = tk.Tk()
    root.withdraw()
    file_path_storge=[]
    print('aaa')
    for j in range(2):
        file_path = filedialog.askopenfilename()
        file_path_storge.append(file_path)
    for i in range(2):
        print(ppo)
        file,dst=input_file(file_path_storge[i], int(i+1))                   # 使用到 input_file 函數
        dim4resizeimg.append(dst)
        inputfile.append(file)
    return inputfile,dim4resizeimg

## 步驟2

輸入*要預測的圖片像素*與*檔案種類*(該函數一張圖片或影片使用一次)
    
    仔入步驟2模型(預測形狀)
    
    預測每張*圖片像素*的形狀
    
    若*檔案種類*不是圖片(是影片)則預測每一張圖片後，每一張最大機率的標籤最多的
    
    若*檔案種類*是圖片則取最大的機率的標籤
    
輸出*預測標籤*與*輸入圖片的預測為該類別的機率?*

ps裡面使用到了 步驟2模型(預測形狀) 

In [5]:
def ring_or_stright(ds, inputfile):
    """
    "圖片的ndarry資料=>prediction[0]=0是直鏈，prediction[0]=1是環"

    ds是四維圖片，inputfile是圖或影片
    dsd是各個類別機率
    prediction[0]是預測出來結果
    prediction[0]=0是直鏈，prediction[0]=1是環
    值鏈編號:0，環編號:1，支鏈編號2
    """
    model = tf.keras.models.load_model(u"C:/D/New/molecule_avo_fig/branch/predict_ring_branch_straight/NN/model/1/want_my_model2.h5")
    # print(ds)
    dsd1 = model.predict(ds)
    # print(dsd.shape,'dsd.shape')  #(13,2)or(1, 2) dsd.shape
    # print(dsd)   # dsd=[[9.99898791e-01 01215606e-04]] dsd=[[0.9965252  0.00347477]]
    prediction = []
    # print(len(dsd[0]),len(dsd[1]))
    dsd=copy.deepcopy(dsd1)
    if inputfile != 'image':
        # print(len(dsd[:,0]),len(dsd[0,:]))
        for k in range(len(dsd[:, 0])):
            for h in range(len(dsd[0, :])):
                # max(dst[k]))，那一張圖片中，10個編號中哪個機率最大，dst[k,h]某個圖片的某個編號機率
                if (dsd[k, h] >= max(dsd[k])) and max(dsd[k]) >= 0.8: 
                    prediction.append([dsd[k, h], h])  # [最大機率，編號]
        a = [i[1] for i in prediction]  # a13張圖片中13個最大機率的編號
        prediction[0] = max(a, key=a.count)  # https://blog.csdn.net/DeniuHe/article/details/103092390  获取list中出现次数最多的元素
    else:
        prediction = np.argmax(dsd, axis=-1)
    #print(prediction[0],dsd1)
    return prediction[0], {'ring': dsd1[0][1], 'line': dsd1[0][0], 'branch': dsd1[0][2]}

## 步驟3 (預測碳數)

#### 直/環/支

In [6]:
def Cnumber(dst, inputfile, structure):
    """
    "環圖片的ndarry資料=>HOMO、LUMO，碳數以及特徵值、特徵向量"

    先預測輸入的四維資料
    根據輸入含有多張截圖圖片或是一張圖片判斷prediction[0]
    prediction[0]=0表示預測為五環
    prediction[0]=1表示預測為六環
    之後放入diffCHOOMOLUMO函數得出五環或是六環在眾多軌域中，HOMO、LUMO，以及特徵值、特徵向量
    """
    # 步驟一前處理
    # dst = cv2.resize(im, (128,128), interpolation=cv2.INTER_AREA)
    # dst = copy.deepcopy(dst[np.newaxis, :])
    # 步驟一
    if structure==0:
        model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/predict_straight_C_number.h5')
        dst = model.predict(dst)
        Probability={'2C': dst[0][0], '4C': dst[0][1], '6C': dst[0][2]}
        def howmuchC(prediction):
            howmuchCnumber=prediction+prediction+2
            return howmuchCnumber
    elif structure==1:
        model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/predict_ring_C_number.h5')
        dst = model.predict(dst)
        Probability={'5C': dst[0][0], '6C': dst[0][1]}
        def howmuchC(prediction):
            howmuchCnumber=prediction+5
            return howmuchCnumber
    else:
        model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/branch/C_number_prediction/NN/模型/!7/want_my_model2.h5')
        dst = model.predict(dst)
        Probability={'6C': dst[0][0], '8C': dst[0][1], '10C': dst[0][2]}
        def howmuchC(prediction):
            howmuchCnumber=prediction+prediction+6
            return howmuchCnumber
    #print(dst.shape, 'dst.shape')  # (13, 10) dst.shape，13張圖片10個類別
    prediction = []
    # print(len(dst[0]),len(dst[1]))
    if inputfile != 'image':
        #print(len(dst[:, 0]), len(dst[0, :]))
        for k in range(len(dst[:, 0])):
            for h in range(len(dst[0, :])):
                if (dst[k, h] >= max(dst[k])) and max(
                        dst[k]) >= 0.8:  # max(dst[k]))，那一張圖片中，10個編號中哪個機率最大，dst[k,h]某個圖片的某個編號機率
                    prediction.append([dst[k, h], h])  # [最大機率，編號]
        a = [i[1] for i in prediction]  # a13張圖片中13個最大機率的編號
        # https://blog.csdn.net/DeniuHe/article/details/103092390   获取list中出现次数最多的元素
        prediction[0] = max(a,key=a.count)  
    else:
        prediction = np.argmax(dst, axis=-1)
    # prediction = model.predict_classes(dst)
    #print(prediction[0])
    howmuchCnumber=howmuchC(prediction[0])
    return howmuchCnumber,Probability

## 步驟4 (Huckel)

單一分子

In [7]:
def huckel(moleculeshape,moleculeCnumber):
    print('碳數與形狀產生huckel矩陣與特徵值特徵向量')
    H=np.zeros([moleculeCnumber,moleculeCnumber]).astype('int64')
    bb=int((moleculeCnumber-4)/2) if moleculeshape==2 else None   # 長鏈中連接支鏈的數量
    b=(moleculeCnumber-bb) if moleculeshape==2  else copy.deepcopy(moleculeCnumber) # 4是支鏈旁的兩個雙鍵，不會有支鏈連接，Lbb事支鏈長的部份的碳數
    connect_position=[]
    for j in range(b-1):
        p1 = j+1
        p2 = j 
        #print(p1, p2)
        H[p1, p2] = 1
        H[p2, p1] = 1
        connect_position.append([p2, p1])
    if moleculeshape==1:
        H[p1, 0] = 1
        H[0, p1] = 1
        connect_position.append([p1, 0])
    if moleculeshape==2:
        for j in range(bb):
            p1 = b+j
            p2 = j+2
            #print(p1, p2)
            H[p1, p2] = 1
            H[p2, p1] = 1
            connect_position.append([p2, p1])
    print(connect_position)
    print(H)
    eigvals, eigvecs = la.eig(H)
    eigvals = eigvals.real
    eigvals=np.around(eigvals,5)
    eigvals_list=eigvals.tolist()
    #klk = bondorder(b,eigvals_list,eigvecs,ring_or_straight)
    print(eigvals,'eigvals')
    print(eigvecs,'eigvecs')
    return eigvals, eigvecs, connect_position

## 步驟5 (預測HOMOLUMO)

### 前處理

輸入*eigvecs*也就是*特徵向量*    單一分子

    第一個for是把每個特徵向量分割，並增加維度，存入vec
    
    第2個for是照順序取vec，再從vec隨機挑兩個，三個連接在一起並存入vvecc
    
輸出*vvecc*三個連接在一起的波函數係數

In [8]:
def Pretreatment5(eigvecs):
    # 加入每個軌域到vec
    vec=[]
    for e in range(len(eigvecs[0])):
        b1 = copy.deepcopy(eigvecs[:, e].reshape(len(eigvecs[0]), 1))
        b1 = b1[:,:, np.newaxis]
        b1 = b1[np.newaxis, :]
        if len(vec) == 0:
            vec = copy.deepcopy(b1)
        else:
            vec = np.concatenate((vec, b1), axis=0)
    # 每個軌域旁都有軌域
    vvecc=[]
    for i in range(len(vec)):  # img=(-1,n,3,1)鬼域
        true_img = copy.deepcopy(vec[i])
        r1 = random.randint(0, (len(vec) - 1))
        r2 = random.randint(0, (len(vec) - 1))
        concat_img = copy.deepcopy(np.concatenate((true_img, vec[r1]), axis=1))
        concat_img = np.concatenate((vec[r2], concat_img), axis=1)
        concat_img = copy.deepcopy(concat_img[np.newaxis, :])
        if len(vvecc) == 0:
            vvecc = copy.deepcopy(concat_img)
        else:
            vvecc = np.concatenate((vvecc, concat_img), axis=0)
    return vvecc
    

### 步驟5模型預測

輸入:*形狀、碳數、特徵向量*

    使用Pretreatment5函數，輸入特徵向量產生三個連接在一起的波函數係數
    
    各種分子模型，使用到*形狀、碳數*
    
    第一個for是算出的特徵向量有可能正負號相反，所以乘-1再預測一次較保險
    
    第2個for是讓所有*特徵向量*進入模型被預測，若預測標籤是0就存入*HOMO*，若預測標籤是1就存入*LUMO*

輸出: *HOMO*，*LUMO*

In [9]:
def HOMOLUMO(shape,Carbonnumber,eigvecs):  
    veccc=Pretreatment5(eigvecs)
    HOMO, LUMO = copy.deepcopy([]), copy.deepcopy([])
    # 決定模型
    if shape==0:
        if Carbonnumber==2:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/2.h5')
        elif Carbonnumber==4:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/4.h5')
        elif Carbonnumber==6:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/6.h5')
        elif Carbonnumber==8:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/8.h5')
        elif Carbonnumber==10:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/10.h5')
        elif Carbonnumber==12:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/12.h5')
        elif Carbonnumber==14:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/14.h5')
        elif Carbonnumber==16:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/16.h5')
        elif Carbonnumber==18:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/18.h5')
        elif Carbonnumber==20:
            model = tf.keras.models.load_model(u'C:/D/gogo/reaction/eigvals, eigvecs/20C，2X3，3，隨機，300，2/20.h5')
        else:
            None
    elif shape==1:
        if Carbonnumber==5:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/5Cring.h5')
        else:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/load_model/load_model_7.1_ring_predict_eig1/6Cring.h5')
    else:
        if Carbonnumber==6:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/branch/C_number_prediction/eigvals, eigvecs,branch/6C，6X3X1，隨機，200/6.h5')
        elif Carbonnumber==8:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/branch/C_number_prediction/eigvals, eigvecs,branch/8C，8X3X1，隨機，200/8.h5')
        else:
            model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/branch/C_number_prediction/eigvals, eigvecs,branch/10C，10X3X1，隨機，200/10.h5')
    # 使用模型
    for j in range(1):
        if j==1:
            veccc=-1*veccc                                       # 算出的特徵向量有可能正負號相反，所以成-1在預測一次較保險
        for i in range(len(veccc)):
            ss = copy.deepcopy(veccc[i])
            ss2 = copy.deepcopy(ss[np.newaxis, :])
            #prediction = model.predict_classes(ss2)
            #print(ss[:, 1, :],'ss[:, 1, :]')
            prediction = copy.deepcopy(model.predict(ss2))
            prediction=np.argmax(prediction,axis=1)
            #print(prediction[0])
            if prediction[0] == 0:
                HOM = copy.deepcopy(ss[:, 1, :])                 # ss[:,1,:]shape是4*1或6*1等
                HOMO.append(HOM)
                print(HOM, '0')
            if prediction[0] == 1:
                LUM = copy.deepcopy(ss[:, 1, :])
                LUMO.append(LUM)
    return HOMO, LUMO

In [10]:
"""
# test
shape=[0,1,2]
LineC=[2,4,6,8,10,12,14,16,18,20]
ringC=[5,6]
branchC=[6,8,10]
for i in LineC:
    a,b,c=huckel(0,i)
    d,e=HOMOLUMO(0,i,b)
    print(np.shape(d),np.shape(e),'0',i)
for i in ringC:
    a,b,c=huckel(1,i)
    d,e=HOMOLUMO(1,i,b)
    print(np.shape(d),np.shape(e),'1',i)
for i in branchC:
    a,b,c=huckel(2,i)
    d,e=HOMOLUMO(2,i,b)
    print(np.shape(d),np.shape(e),'2',i)
"""

"\n# test\nshape=[0,1,2]\nLineC=[2,4,6,8,10,12,14,16,18,20]\nringC=[5,6]\nbranchC=[6,8,10]\nfor i in LineC:\n    a,b,c=huckel(0,i)\n    d,e=HOMOLUMO(0,i,b)\n    print(np.shape(d),np.shape(e),'0',i)\nfor i in ringC:\n    a,b,c=huckel(1,i)\n    d,e=HOMOLUMO(1,i,b)\n    print(np.shape(d),np.shape(e),'1',i)\nfor i in branchC:\n    a,b,c=huckel(2,i)\n    d,e=HOMOLUMO(2,i,b)\n    print(np.shape(d),np.shape(e),'2',i)\n"

## 步驟6 (預測反應產物)

###前處理



算階層

In [11]:
def x(num):  # https://www.runoob.com/python3/python3-factorial.html
    #print(num,'輸入')
    factorial = 1
    for i in range(1,num + 1):
        factorial = factorial*i
    #print(factorial,'輸出')
    return factorial

輸入長度輸出所有可能的兩個或多個數字，不重複，兩個或多個數字顛倒也不重複

輸入:*Cnumber*,*rnumber*，也就是碳數與要件節的數量(要預測會不會反應的位置數量)(實際上是2)
    
    算碳數的階層，知道從鬼域中取2，有多少數量
    
    while是若真的取道這數量就離開
    
        for 是隨機取2個炭的位置，加入rr
        
        若排列順序後不在r裡，若rr裡的數字不重複，就加入*r*
        
輸出: *r*，也就是所有可能鍵結的位置

In [12]:
def randomnumber(Cnumber,rnumber):
    pn =x(Cnumber)/((x(Cnumber-rnumber))*x(rnumber))
    #print(pn,x(Cnumber))
    ppnn=0
    r=[]
    while ppnn != pn:
        # 隨機選位置
        rr=copy.deepcopy([])
        for i in range(rnumber):
            rr.append(random.randint(0,Cnumber-1))
       
        if (sorted(rr) not in r) and (len(set(rr))==rnumber):
            r.append(sorted(rr))
            ppnn+=1
    #print(r)
    return r

輸入鬼域輸出隨機位置的鬼域

In [13]:
def sigmaMO(omo):
    rgbHOMO=[]
    for j in range(len(omo)):        # 環可能有兩個HOMO/LUMO
        mo=copy.deepcopy(omo[j])
        randomnlits = copy.deepcopy(randomnumber(len(mo),2))
        print(len(randomnlits))
        #print(randomnlits)
            # 交換數值
        for i in range(len(randomnlits)):  # len(randomnlits)是從n個C取2的數量，隨機選擇位置的數量，randomnlits是隨機選的位置[[1,2],[3,4]...]
            # 建立通道
            g=copy.deepcopy(np.zeros(len(mo)).reshape(len(mo),1))
            bl = copy.deepcopy(np.zeros(len(mo)).reshape(len(mo),1))
            re=copy.deepcopy(mo.reshape(len(mo),1))
            #print(r)              # randomnlitsplusmines是紀錄位置及該位置的波函數正負號
            g[randomnlits[i][0]] = copy.deepcopy(re[randomnlits[i][0]])
            g[randomnlits[i][1]] = copy.deepcopy(re[randomnlits[i][1]])
            re[randomnlits[i][0]] = copy.deepcopy(0)
            re[randomnlits[i][1]] = copy.deepcopy(0)
            g=copy.deepcopy(g[:,:,np.newaxis])
            bl=copy.deepcopy(bl[:,:,np.newaxis])
            re=copy.deepcopy(re[:,:,np.newaxis])
            gg = []
            # n變成20
            for i in [re,g,bl]:
                #print(c,'c')
                while len(i)<20:
                    i = np.insert(i, 0, np.zeros(1), 0)
                    if len(i)<20:
                        i = np.insert(i, np.size(i,0), np.zeros(1), 0)
                    else:
                        break
                gg.append(i)
            re = copy.deepcopy(gg[0])
            g = copy.deepcopy(gg[1])
            bl = copy.deepcopy(gg[2])
            reg = np.append(re, g, axis=2)  #　（：：在這裡合併）
            regbl = np.append(reg, bl, axis=2)
            regbl = copy.deepcopy(regbl[np.newaxis, :])
            if len(rgbHOMO)==0:
                rgbHOMO = copy.deepcopy(regbl)
            else:
                rgbHOMO = copy.deepcopy(np.concatenate((rgbHOMO, regbl), axis=0))
    print(np.shape(rgbHOMO))
    return rgbHOMO

4+2的HOMOLUMO

In [14]:
def four_two_homo_lumo(rgbH,rgbL):
    print('1是會反映/0是不會反應')
    data0=[]
    for i in range(len(rgbH)):  # 判斷label
        for j in range(len(rgbL)):
            for k in range(2):
                if k==0:       # 若LUMO沒反轉
                    #print(i,j)
                    #print(rgbL[j])
                    reaction_rgb = copy.deepcopy(np.append(rgbH[i], rgbL[j], axis=1))  # HOMOLUMO合併
                    data0.append(reaction_rgb)
                else:         # 若LUMO反轉
                    #print(rgbL[j][::-1])
                    if np.allclose(rgbL[j][::-1],rgbL[j])==False:
                        reaction_rgb = copy.deepcopy(np.append(rgbH[i], rgbL[j][::-1], axis=1))  # HOMOLUMO合併
                        data0.append(reaction_rgb)
    #print(data0[0])
    return data0

4+4/2+2的LUMOLUMO

In [15]:
def two_two_four_four_lumo_lumo(rgbL1,rgbL2):
    print('4+4/2+2的LUMOLUMO，1是會反映/0是不會反應')
    data0=[]
    print(len(rgbL1),'len(rgbL1)')
    print(len(rgbL2),'len(rgbL2)')
    for i in range(len(rgbL1)):  # 判斷label
        for j in range(len(rgbL2)):
            for k in range(2):
                if k==0:       # 若LUMO沒反轉
                    reaction_rgb = copy.deepcopy(np.append(rgbL1[i], rgbL2[j], axis=1))  # LUMOLUMO合併
                    data0.append(reaction_rgb)
                else:         # 若LUMO反轉
                    if np.allclose(rgbL2[j][::-1],rgbL2[j])==False:
                        reaction_rgb = copy.deepcopy(np.append(rgbL1[i], rgbL2[j][::-1], axis=1))  # LUMOLUMO合併
                        data0.append(reaction_rgb)

    return data0

全

In [16]:
def all(HOMO_1,LUMO_1,HOMO_2,LUMO_2):
    rgbH_1=sigmaMO(HOMO_1)
    rgbL_1=sigmaMO(LUMO_1)
    rgbH_2=sigmaMO(HOMO_2)
    rgbL_2=sigmaMO(LUMO_2)
    #print(np.shape(rgbH_1),'np.shape(rgbH_1)')
    #print(rgbH_1[0],'rgbH_1[0]')
    #print(np.shape(rgbL_2),'np.shape(rgbL_2)')
    #print(rgbL_2[0],'rgbL_2[0]')
    data42,data2244=[],[]
    combine_H1L2=copy.deepcopy(four_two_homo_lumo(rgbH_1,rgbL_2))       # 兩個資料量一樣，因為包含會反應的與不會反應的
    combine_H2L1=copy.deepcopy(four_two_homo_lumo(rgbH_2,rgbL_1))
    combine_L2L1=copy.deepcopy(two_two_four_four_lumo_lumo(rgbL_2,rgbL_1))
    combine_L1L2=copy.deepcopy(two_two_four_four_lumo_lumo(rgbL_1,rgbL_2))
    print(np.shape(combine_H1L2),'np.shape(combine_H1L2)')
    print(np.shape(combine_H2L1),'np.shape(combine_H2L1)')
    print(np.shape(combine_L2L1),'np.shape(combine_L2L1)')
    print(np.shape(combine_L1L2),'np.shape(combine_L1L2)')
    data42+=combine_H1L2
    data42+=combine_H2L1
    data2244+=combine_L2L1
    data2244+=combine_L1L2
    print(np.shape(data42),np.shape(data2244),'np.shape(data42),np.shape(data2244)')
    return data42, data2244

### 步驟6模型預測

從編編到有數字的地方，可以判別矩陣

In [17]:
def step6(HOMO_1,LUMO_1,HOMO_2,LUMO_2):
    getdata42, getdata2244=all(HOMO_1,LUMO_1,HOMO_2,LUMO_2)
    canreactiondata42, canreactiondata2244=[],[]
    print(getdata42[0],'getdata42[0]')
    print(np.shape(getdata42[0][np.newaxis, :]),'np.shape(getdata42[0])')
    print(type(getdata42[0]),'type(getdata42[0])')
    model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/branch/can_reaction_ring_branch_straight/42平均在trainvalidtest_加熱_300000_0.6_X3_1429628_10152/哦型/want2.h5')
    #prediction = model.predict(getdata42)
    #prediction=np.argmax(prediction,axis=1)
    #print(prediction)
    for i in range(len(getdata42)):
        prediction = copy.deepcopy(model.predict(getdata42[i][np.newaxis, :]))
        prediction=np.argmax(prediction,axis=1)
        if prediction[0]==1:
            #print(np.shape(getdata42[i]))
            boundary=[]
            for j in range(20):
                if getdata42[i][:,0,0][j]!=0:
                    print(np.shape(getdata42[i][:,0,0][j]),'np.shape(getdata42[i][:,0,0][j])')
                    boundary.append(j)
"""
            for j in range(20):
                if getdata42[i][(19-j)]!=0:
                    boundary.append((19-j))
            
            
                        
        canreactiondata42.append(getdata42[i])
    model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/branch/can_reaction_ring_branch_straight/4422平均在trainvalidtest_加光_300000_0.6_X3_1429628_10152/!5.h5')
    for i in range(len(getdata4422)):
        prediction = copy.deepcopy(model.predict(getdata4422[i]))
        prediction=np.argmax(prediction,axis=1)
        canreactiondata4422.append(getdata4422[i])
"""

"\n            for j in range(20):\n                if getdata42[i][(19-j)]!=0:\n                    boundary.append((19-j))\n            \n            \n                        \n        canreactiondata42.append(getdata42[i])\n    model = tf.keras.models.load_model(u'C:/D/New/molecule_avo_fig/branch/can_reaction_ring_branch_straight/4422平均在trainvalidtest_加光_300000_0.6_X3_1429628_10152/!5.h5')\n    for i in range(len(getdata4422)):\n        prediction = copy.deepcopy(model.predict(getdata4422[i]))\n        prediction=np.argmax(prediction,axis=1)\n        canreactiondata4422.append(getdata4422[i])\n"

最後要執行所有函數

In [18]:
def clicked(arg):
    with out:
        clear_output()
        inputfile,dim4resizeimg=frf('ppo')
        
        # 步驟2
        moleculeshape1,a = ring_or_stright(dim4resizeimg[0], inputfile[0])  # (第一個檔案(圖或影)) ringorstright1=0是直鏈，ringorstright1=1是環
        moleculeshape2,b = ring_or_stright(dim4resizeimg[1], inputfile[1])  # (第二個檔案(圖或影)) ringorstright2=0是直鏈，ringorstright2=1是環
        print(moleculeshape1,'moleculeshape1')
        print(a,'a')
        print(moleculeshape2,'moleculeshape2')
        print(b,'b')
        
        # 步驟3
        moleculeCnumber1,a=Cnumber(dim4resizeimg[0], inputfile[0],moleculeshape1)   # 輸入 圖影片1像素 檔案1種類 步驟2結果
        moleculeCnumber2,b=Cnumber(dim4resizeimg[1], inputfile[1],moleculeshape2)   # 輸入 圖影片2像素 檔案2種類 步驟2結果
        print(moleculeCnumber1,'moleculeCnumber1')
        print(a,'a')
        print(moleculeCnumber2,'moleculeCnumber2')
        print(b,'b')
        
        # 步驟4
        eigvals1, eigvecs1, connect_position1=huckel(moleculeshape1,moleculeCnumber1)
        eigvals2, eigvecs2, connect_position2=huckel(moleculeshape2,moleculeCnumber2)
        
        # 步驟5
        HOMO1,LUMO1=HOMOLUMO(moleculeshape1,moleculeCnumber1,eigvecs1)
        HOMO2,LUMO2=HOMOLUMO(moleculeshape2,moleculeCnumber2,eigvecs2)
        print(HOMO1,'HOMO1')
        print(HOMO2,'HOMO2')
        print(LUMO1,'LUMO1')
        print(LUMO2,'LUMO2')
        
        # 步驟6
        step6(HOMO1,LUMO1,HOMO2,LUMO2)
        #data42, data2244=all(HOMO1,LUMO1,HOMO2,LUMO2)
        #print(data42[0],'data42[0]')
        #print(data2244[0],'data2244[0]')
        
        

最後buttom所需

In [19]:
button_download = widgets.Button(description = 'Test Button')   
button_download.on_click(clicked)
page = widgets.VBox([button_download,out])
display(page)#button_download